In [222]:
import ipynb.fs.full.get_artists_info_api as getter

In [38]:
tracks_id = []
for i in range(len(getter.tracks)):
    tracks_id.append(getter.tracks[i]['id'])

tracks_id = list(dict.fromkeys(tracks_id))

In [111]:
track_features_list = []

for i in range(len(tracks_id)):
    track_features_list.append(getter.spotify.audio_features(tracks_id[i]))

In [112]:
#there are some tracks without their audio features so we need to remove them from the list
track_features_list = list(filter(lambda x: x != [None], track_features_list))

In [113]:
for i in range(len(track_features_list)):
    track_features_list[i] = dict(list(filter(lambda x: x[0] == 'id' or type(x[1]) != str, list(track_features_list[i][0].items()))))

In [164]:
artists = list(filter(lambda x: x['genres'] != '' ,getter.artists))

In [169]:
genres_list = []

for i in range(len(artists)):
    keys = ['id', 'genre']
    genre = dict.fromkeys(keys)
    for j in range(len(getter.tracks)):
        if artists[i]['id'] == getter.tracks[j]['artists'][0]['id']:
            genre['id'] = getter.tracks[j]['id']
            genre['genre'] = artists[i]['genres'].split(', ')
            genre['genre'] = genre['genre'][0]
            genres_list.append(genre)

In [196]:
for i in range(len(track_features_list)):
    for j in range(len(genres_list)):
        if track_features_list[i]['id'] == genres_list[j]['id']:
            track_features_list[i]['genre'] = genres_list[j]['genre']

In [201]:
track_features_list = list(filter(lambda x: len(x) == 15, track_features_list))

---------------------------------------------------------------------------------------------------------------

In [245]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [224]:
df = pd.DataFrame(track_features_list)

In [225]:
df.insert(0, 'id', df.pop('id'))

In [226]:
df.head(10)

,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,genre
0,3HWcK5EIfIdOQD1Rsj4yek,0.747,0.422,7,-12.890,0,0.4970,0.00522,0.000831,0.1190,0.125,132.128,176364,4,plugg
1,2lFeir1XMPIRPcvNGQwCSr,0.845,0.483,2,-7.859,1,0.2770,0.01090,0.000000,0.1360,0.643,129.049,98605,4,pop rap
2,2F1Z3G7IZKissAgLtZSUup,0.629,0.726,6,-10.098,0,0.3310,0.18600,0.000000,0.2440,0.839,81.148,114000,4,christian trap
3,4RBiD2UN3aDpkcV7KX0S0N,0.623,0.668,6,-7.282,0,0.0531,0.47500,0.000000,0.0862,0.823,161.948,183067,4,canadian pop
4,6LiCDjlGCBx8DK4VhOijvp,0.744,0.680,8,-6.866,1,0.2320,0.23800,0.000000,0.1070,0.507,112.873,167030,4,new romantic
5,3JcKCicqIJ0qYWnHTLQzN9,0.601,0.713,4,-3.758,0,0.0449,0.02820,0.000000,0.1580,0.464,122.872,169756,4,big room
6,4u2gL1g6AgF2vlAURETOXi,0.429,0.834,0,-6.610,0,0.2610,0.01060,0.000000,0.1630,0.664,179.932,110517,4,glitchcore
7,6ygf36lTU1bMocl46hZOAG,0.680,0.597,1,-6.627,1,0.0425,0.02980,0.000002,0.0777,0.527,155.980,251429,4,chilean indie
8,1G9udmctlnLyk5BXgJbifd,0.438,0.178,5,-9.402,1,0.0414,0.95700,0.000001,0.3160,0.344,75.250,170964,1,dance pop
9,50fqhNXa174Te7j95WRQHT,0.710,0.625,6,-9.008,0,0.0295,0.02900,0.000011,0.1080,0.784,140.994,307014,4,dance pop


In [227]:
model = df.drop(['id'], axis=1)

In [228]:
label = model.iloc[:, -1]
label.head(10)

0             plugg
1           pop rap
2    christian trap
3      canadian pop
4      new romantic
5          big room
6        glitchcore
7     chilean indie
8         dance pop
9         dance pop
Name: genre, dtype: object

In [232]:
encoder = LabelEncoder()
y = encoder.fit_transform(label)
y

array([130, 135,  51,  43, 123,  33,  91,  48,  64,  64, 110, 129, 133,
        15, 105, 133,  38,  64,  20,  64,  64,  93,  17, 133,  39,  64,
        82,  43, 124,  97,  64,  68,  41,  26,  17,  42,  59,  87,  19,
       113,  72,   6, 140,  89, 102,  60,  33,  90,  12,  65,  64,  15,
        64, 133, 106,  64,  18,  64,  55, 127,  67,  82,  66,  64,  42,
        18,  40, 131, 110,  64,  64, 110, 130, 143, 141,  40,   9, 122,
       147,   9,  32,  27,  81,  64, 121, 105,  64, 142,   3,  79,  64,
        86, 134,   5, 136,  23,  14,  64, 115,  22,  52,  64,  22,  78,
       120, 112,  75,   6,  64,  91,  17,  12,  56, 133,  93,   6, 118,
         2,  15,  43,  53,  64,  10,  64,  44,  29,  43,   2, 138, 119,
        92,  34,  64,  83,  85,  64,  41,  64,  43, 126,  29,  64, 125,
        43,  26,   5,  64,  12,  22,  12,  10,  60, 102,  64, 123, 137,
        12,  60,   4,   6,  76,  64,   7,  53,   2, 107,  60, 133,  80,
        10,   2, 109,  10,  83,  60,  61, 108,  10,  40,  62,  9

In [239]:
scaler = StandardScaler()
x = scaler.fit_transform(np.array(model.iloc[:, :-1], dtype = float))
x.shape

(294, 13)

In [243]:
x_train, y_train, x_test, y_test = train_test_split(x, y, test_size=0.3)